In [1]:
import pandas as pd
import nltk
import gensim
from gensim import downloader
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from scipy.spatial import distance
import re
import numpy as np

glove_vectors = gensim.downloader.load('word2vec-google-news-300')
stop_words = set(stopwords.words('english'))

In [2]:
import torch
from transformers import BertTokenizer, BertModel
import logging
import pandas as pd
import matplotlib.pyplot as plt
from textwrap import wrap
import numpy as np
from sklearn.manifold import TSNE
from sentence_transformers import SentenceTransformer
from sklearn.cluster import KMeans
from collections import Counter
from functools import reduce

 
num_topics = 50
model_bert = SentenceTransformer('bert-base-nli-max-tokens')               

: 

: 

In [ ]:
df_books = pd.read_csv("./raw_data/Books.csv")
df_summary = pd.read_csv('booksummaries.txt', sep = '\t')
df_summary.columns = ['Code1', 'Code2', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Code3', 'Book-Summary']

In [ ]:
def Compute_ISBN_to_Title(df_merged):
    ISBN_to_Title = {}
    for row in df_merged.index:
        title = df_merged["Book-Title"][row]
        isbn = df_merged["ISBN"][row]
        ISBN_to_Title[isbn] = title
    return ISBN_to_Title

def merge_book_and_summary(df_summary, df_books):
    df_merged = pd.merge(df_summary, df_books, how='inner', on =['Book-Title'])
    df_merged = df_merged.drop(['Code1','Code2','Code3','Publisher', 'Year-Of-Publication_x', 'Image-URL-S', 'Image-URL-M', 'Image-URL-L', 'Book-Author_y', 'Year-Of-Publication_y'], axis=1)
    df_merged = df_merged[df_merged['Book-Title'] != 'Deathstalker Rebellion']
    return df_merged

df_merged = merge_book_and_summary(df_summary, df_books).drop_duplicates(subset=['Book-Title'])
#df_merged.to_csv('Books_Summary_merged.csv')

In [ ]:
def all_books_to_vecs(books):
  embedding_bert = np.array(model_bert.encode(books))
  return embedding_bert

def reduce_tsne(embedding):
    tsne = TSNE(n_components=2)
    reduced = tsne.fit_transform( embedding )
    
    return reduced

def predict_topics_with_kmeans(embeddings,num_topics):
  kmeans_model = KMeans(num_topics)
  kmeans_model.fit(embeddings)
  topics_labels = kmeans_model.predict(embeddings)
  return topics_labels

df_summary_bert = df_merged
all_summary_list = df_summary_bert["Book-Summary"].tolist()
all_summary_embeddings = all_books_to_vecs(all_summary_list)
df_summary["vector"] = all_summary_embeddings.tolist()

In [ ]:
#Visualize
def plot_embeddings(embedding, labels,title):

    labels = np.array( labels )
    distinct_labels =  set( labels )
    
    n = len(embedding)
    counter = Counter(labels)
    for i in range(len( distinct_labels )):
        ratio = (counter[i] / n )* 100
        cluster_label = f"cluster {i}: { round(ratio,2)}"
        x = embedding[:, 0][labels == i]
        y = embedding[:, 1][labels == i]
        plt.plot(x, y, '.', alpha=0.4, label= cluster_label)
    plt.legend(title="Topic",loc = 'upper left', bbox_to_anchor=(1.01,1))
    plt.title(title)

embedding_bert_tsne =  reduce_tsne(all_summary_embeddings)
labels_bert_tsne  = predict_topics_with_kmeans(embedding_bert_tsne,num_topics)
df_summary_bert["group"] = labels_bert_tsne
df_summary_bert["vector"] = all_summary_embeddings.tolist()
#plot_embeddings(embedding_bert_tsne,labels_bert_tsne,"Bert with T-sne")

In [ ]:
def compute_Book2Vec(df_merged):
    title_dict = {}
    summary_list = []

    for row in df_merged.index:
        title = df_merged['Book-Title'][row]
        summary = df_merged['Book-Summary'][row]
        summary_nonum = re.sub(r'\d+', '', summary) #remove number
        tokenizer = RegexpTokenizer(r'\w+') #remove punctuation
        summary_tokens = tokenizer.tokenize(summary_nonum) 
        filtered_summary_tokens = [w for w in summary_tokens if not w.lower() in stop_words] #remove stop words

        summary_vectors = np.zeros(300)
    
        n = 0
        for summary_token in filtered_summary_tokens:
            try:
                summary_token_vec = glove_vectors[summary_token]
            except:
                summary_token_vec = np.zeros(300)
            summary_vectors += summary_token_vec
            n += 1
        summary_list.append (summary_vectors / n)
    
    return summary_list

summary_list = compute_Book2Vec(df_summary_bert)
df_summary = df_summary.drop(['Unnamed: 0.1', 'Unnamed: 0'], axis=1)
df_summary = df_summary.rename(columns={"vector": "BERT"})
df_summary["word2vec"] = summary_list

In [ ]:
df_summary.to_csv("book-vec-group.csv")